In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-basis-constructor

# Einfüehrung en Bruchteil-Gates

*Verbruchsschätzung: ungefähr 30 Sekunde op enem Heron r2 Prozessor (OPJEPASS: Dat es bloß en Schätzung. Ding Laufzick kann anders sin.)*
## Hintergrund
### Bruchteil-Gates op IBM QPUs
Bruchteil-Gates sinn parametrisierte Quante-Gates die et möglich maache arbiträr-Winkel Rotationen direkt ußzeföhre (en bestemmpte Jränze),
un dodurch bruchme se nit en miehr Basis-Gates zerrleeje.
Door dat mer de nativ Interakzjohne zwesche de physikalische Qubits nütze, künne Benutzer bestemmpte Unitäre effizienter op Hardware implementiere.

IBM Quantum&reg; Heron QPUs ongerstötze die folgende Bruchteil-Gates:
- $R_{ZZ}(\theta)$ för $0 < \theta < \pi / 2$
- $R_X(\theta)$ för all reell Wäät $\theta$

Die Gates künne suwohl de Deef wie och de Dauer vun Quanteschaltkreis bedütend reduziere.
Se sinn besönders vorteilhaft en Aanwendunge die stark op $R_{ZZ}$ un $R_X$ baue,
wie Hamiltonscher Simulation, dem Quantum Approximate Optimization Algorithm (QAOA), un Quante-Kernel Methode.
En däm Tutorial konzentriere mer uns op der Quante-Kernel als praktisch Beispill.
### Beschränkunge
Bruchteil-Gates sinn em Momang en experimentell Funkzjon un komme met e paar Beschränkunge:
- $R_{ZZ}$ es beschränk op Winkel em Bereich $0 < \theta < \pi / 2$.
- De Verweendung vun Bruchteil-Gates weed nit ongerstötz för [dynamische Schaltkreis](/guides/classical-feedforward-and-control-flow), [Pauli twirling](/guides/error-mitigation-and-suppression-techniques#pauli-twirling), [probabilistische Fehlerauslöschung](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) (PEC), un [zero-noise extrapolation](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) (ZNE) (met [probabilistische Fehlerverstärkung](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) (PEA)).

Bruchteil-Gates bruche en andre Workflow als de Standardaansatz.
Dat Tutorial erklärt wie mer met Bruchteil-Gates dörch en praktisch Aanwendung ärbeide.

Luurt ens en die folgend för mieh Details övver Bruchteil-Gates.
- [Bruchteil-Gates](/guides/fractional-gates)
- [Wann mer Bruchteil-Gates *nit* bruche soll](/guides/fractional-gates#when-not-to-use)
## Övverbleck
De Workflow för de Verweendung vun Bruchteil-Gates folgk jenerell dem [Qiskit patterns](/guides/intro-to-patterns) Workflow.
Dä wesentlich Ongerscheid es dat all RZZ Winkel de Beschränkung $0 < \theta \leq \pi/2$ erfülle mösse.
Et jitt zwei Aansätze öm sescher ze stelle dat die Bedingung erföllt weed.
Dat Tutorial konzentreert sich op un empfehlt dä zweete Aansatz.

### 1. Parameterwääte jeneriere die de RZZ Winkelbeschränkung erfülle
Wann de secher bes dat all RZZ Winkel em jöltije Bereich ligge, kanns de dem Standardworkflow vun Qiskit patterns folgke.
En däm Fall jifs de einfach de Parameterwääte als Deil vun enem PUB aan. Dä Workflow löppt esu:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import unitary_overlap
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

Wann de versöchs en PUB aanzejevve dat enn RZZ Gate met enem Winkel ußerhalv vum jöltije Bereich enthällt, kriss de en Fehlermeldung wie:
```
'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (20.0) outside of this range has been requested; via parameter value(s) γ[0]=10.0, substituted in parameter expression 2.0*γ[0].'
```
Öm dä Fehler ze vermeide, solls de dä zweete Aansatz övverleeje dä unge beschrevve weed.
### 2. Parameterwääte aan Schaltkreis zuweise vör de Transpilazjon
Et `qiskit-ibm-runtime` Paket stellt enne spezialisierte Transpiler-Pass bereet dä [`FoldRzzAngle`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle) heißt.
Dä Pass transformeert Quanteschaltkreis esu dat all RZZ Winkel met de RZZ Winkelbeschränkung övvereinstemme.
Wann de et Backend aan `generate_preset_pass_manager` oder `transpile` jiss, wend Qiskit automatisch `FoldRzzAngle` op de Quanteschaltkreis aan.
Dat froog dich drüm Parameterwääte aan Quanteschaltkreis vör de Transpilazjon zuzuweise.
Dä Workflow löppt esu:

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)  # backend should be a heron device or later
backend_name = backend.name
backend_c = service.backend(backend_name)  # w/o fractional gates
backend_f = service.backend(
    backend_name, use_fractional_gates=True
)  # w/ fractional gates
print(f"Backend: {backend_name}")
print(f"No fractional gates: {backend_c.basis_gates}")
print(f"With fractional gates: {backend_f.basis_gates}")
if "rzz" not in backend_f.basis_gates:
    print(f"Backend {backend_name} does not support fractional gates")

Backend: ibm_fez
No fractional gates: ['cz', 'id', 'rz', 'sx', 'x']
With fractional gates: ['cz', 'id', 'rx', 'rz', 'rzz', 'sx', 'x']


Merk dat dä Workflow mieh Berechnungskooste verursaach als dä eeschte Aansatz, weil et Parameterwääte aan Quanteschaltkreis zuwise un de parameterjebondene Schaltkreis lokal speichere beinhällt.
Zosätzlich jitt et enn bekannt Problem en Qiskit woh de Transformazjon vun RZZ Gates en bestemmpte Szenarien fählschloh kann. För en Lösung, luurt bes en dä [Troubleshooting](#troubleshooting) Abschnett.
Dat Tutorial zeigg wie mer Bruchteil-Gates övver dä zweete Aansatz bruche, dörch en Beispill dat vum Quante-Kernel Methode inspireet es.
Öm besser ze verstohn wo Quante-Kernels wohrscheinlich nützlich sinn, empfehle mer dat Lese vun [Liu, Arunachalam & Temme (2021).](https://www.nature.com/articles/s41567-021-01287-z)

De kanns och et [Quantum kernel training](/tutorials/quantum-kernel-training) Tutorial un de [Quantum kernels](/learning/courses/quantum-machine-learning/quantum-kernel-methods) Lekzjon em Quantum machine learning Kurs op IBM Quantum Learning duurchärbeide.
### Aanforderunge
Bevör de met däm Tutorial aanfängst, versichre dat de et Folgend installeert häs:
- Qiskit SDK v2.0 oder späder, met [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) Ongerstötzung
- Qiskit Runtime v0.37 oder späder (`pip install qiskit-ibm-runtime`)
- Qiskit Basis Constructor (`pip install qiskit_basis_constructor`)
### Oprüstung

In [3]:
optimization_level = 2
shots = 2000
reps = 3
rng = np.random.default_rng(seed=123)

In [4]:
def my_zz_feature_map(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    x = ParameterVector("x", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for k in range(reps):
        K = k * num_qubits
        for i in range(num_qubits):
            qc.rz(x[i + K], i)
        pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for i, j in pairs[0::2] + pairs[1::2]:
            qc.rzz((np.pi - x[i + K]) * (np.pi - x[j + K]), i, j)
    return qc


def quantum_kernel(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    qc = my_zz_feature_map(num_qubits, reps=reps)
    inner_product = unitary_overlap(qc, qc, "x", "y", insert_barrier=True)
    inner_product.measure_all()
    return inner_product


def random_parameters(inner_product: QuantumCircuit) -> np.ndarray:
    return np.tile(rng.random(inner_product.num_parameters // 2), 2)


def fidelity(result) -> float:
    ba = result.data.meas
    return ba.get_int_counts().get(0, 0) / ba.num_shots

Quantum kernel circuits and their corresponding parameter values are generated for systems with 4 to 40 qubits, and their fidelities are subsequently evaluated.

In [5]:
qubits = list(range(4, 44, 4))
circuits = [quantum_kernel(i, reps=reps) for i in qubits]
params = [random_parameters(circ) for circ in circuits]

## Workflow met Bruchteil-Gates
### Schrett 1: Klassisch Eingabe op Quanteproblem abbeelde
#### Quante-Kernel Schaltkreis
En däm Abschnett ongersöche mer dä Quante-Kernel Schaltkreis met RZZ Gates öm dä Workflow för Bruchteil-Gates vörzestelle.

Mer fange aan endem mer enne Quanteschaltkreis bouwe öm einzeln Eenträg vun de Kernel-Matrix ze berechne.
Dat weed jemaat endem mer ZZ Feature-Map Schaltkreis met enem unitary overlap kombiniere.
De Kernel-Funkzjon nimmp Vektore em feature-mapped Raum un jitt ehr inner Produkt als Eentraach vun de Kernel-Matrix zeröck:
$$K(x, y) = \langle \Phi(x) | \Phi(y) \rangle,$$
woh $|\Phi(x)\rangle$ dä feature-mapped Quantezustand darstellt.

Mer bouwe manuell enne ZZ Feature-Map Schaltkreis met RZZ Gates.
Obwohl Qiskit enne injebouwte `zz_feature_map` bereet stellt, ongerstötz et zurzigg kein RZZ Gates ab Qiskit v2.0.2 ([luurt bes öm Issue](https://github.com/Qiskit/qiskit/issues/14469)).

Als nächste berechne mer de Kernel-Funkzjon för identisch Eingabe - för Beispill, $K(x, x) = 1$.
Op verrauschte Quantecomputer kann dä Wäät weniger als 1 sin wejen Rausche.
En Resultat nöcher aan 1 bedügg weniger Rausche bei de Ußföhrung.
En däm Tutorial nenne mer dä Wäät de *Fidelity*, defeneert als
$$\text{fidelity} = K(x, x).$$

In [6]:
circuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif" alt="Output of the previous code cell" />

In the standard Qiskit patterns workflow, parameter values are typically passed to the Sampler or Estimator primitive as part of a PUB.
However, when using a backend that supports fractional gates, these parameter values must be explicitly assigned to the quantum circuit prior to transpilation.

In [7]:
b_qc = [
    circ.assign_parameters(param) for circ, param in zip(circuits, params)
]
b_qc[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif" alt="Output of the previous code cell" />

Quante-Kernel Schaltkreis un ehr entsprechend Parameterwääte weede för Systeme met 4 bes 40 Qubits jenereet, un ehr Fidelities weede anschließend bewertet.

In [8]:
backend_f = service.backend(name=backend_name, use_fractional_gates=True)
# pm_f includes `FoldRzzAngle` pass
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_f
)

In [9]:
t_qc_f = pm_f.run(b_qc)
print(t_qc_f[0].count_ops())
t_qc_f[0].draw("mpl", fold=-1)

OrderedDict([('rz', 35), ('rzz', 18), ('x', 13), ('rx', 9), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif)

Em Standard Qiskit patterns Workflow weede Parameterwääte normalerwies als Deil vun enem PUB aan et Sampler oder Estimator Primitive övverjoffe.
Wann mer ävver en Backend bruche dat Bruchteil-Gates ongerstötz, mösse die Parameterwääte explizit vör de Transpilazjon aan dä Quanteschaltkreis zujewise weede.

In [10]:
nnl_f = [qc.num_nonlocal_gates() for qc in t_qc_f]
depth_f = [qc.depth() for qc in t_qc_f]
duration_f = [
    qc.estimate_duration(backend_f.target, unit="u") for qc in t_qc_f
]

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif)

### Schrett 2: Problem för Quante-Hardware Ußföhrung optimiere

Mer transpiliere dann dä Schaltkreis met dem Pass Manager noh dem Standard Qiskit Pattern.
Endem mer en Backend bereet stelle dat Bruchteil-Gates ongerstötz aan `generate_preset_pass_manager`, weed enne spezialiseerde Pass jenanndt `FoldRzzAngle` automatisch enjefögg.
Dä Pass modifizeert dä Schaltkreis öm met de RZZ Winkelbeschränkunge övverein ze stemme.
Als Resultat weede RZZ Gates met negative Wääte em vörijje Beld en positiv Wääte transformeert, un paar zosätzlich X Gates weede enjefögg.

In [11]:
sampler_f = SamplerV2(mode=backend_f)
sampler_f.options.dynamical_decoupling.enable = True
sampler_f.options.dynamical_decoupling.sequence_type = "XY4"
sampler_f.options.dynamical_decoupling.skip_reset_qubits = True

In [12]:
job = sampler_f.run(t_qc_f, shots=shots)
print(job.job_id())

d4bninsi51bc738j97eg


### Step 4: Post-process and return result in desired classical format

You can obtain the kernel function value $K(x, x)$ by measuring the probability of the all-zero bitstring `00...00` in the output.

In [13]:
# job = service.job("d1obougt0npc73flhiag")
result = job.result()
fidelity_f = [fidelity(result=res) for res in result]
print(fidelity_f)
usage_f = job.usage()

[0.9005, 0.647, 0.3345, 0.355, 0.3315, 0.174, 0.1875, 0.149, 0.1175, 0.085]


![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif)

Öm de Einfloß vun Bruchteil-Gates ze bewätte, evaluere mer de Zahl vun Nit-Lokal Gates (CZ un RZZ för dat Backend),
zosamme met Schaltkreisdeefe un Dauerijkeete, un verglieche die Metrike met däne vun enem Standardworkflow späder.

In [14]:
# step 1: map classical inputs to quantum problem
# `circuits` and `params` from the previous section are reused here

In [15]:
# step 2: optimize circuits
backend_c = service.backend(backend_name)  # w/o fractional gates
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc_c = pm_c.run(circuits)
print(t_qc_c[0].count_ops())
t_qc_c[0].draw("mpl", fold=-1)

OrderedDict([('rz', 130), ('sx', 80), ('cz', 36), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a10f2d95-1.avif" alt="Output of the previous code cell" />

In [16]:
nnl_c = [qc.num_nonlocal_gates() for qc in t_qc_c]
depth_c = [qc.depth() for qc in t_qc_c]
duration_c = [
    qc.estimate_duration(backend_c.target, unit="u") for qc in t_qc_c
]

In [17]:
# step 3: execute
sampler_c = SamplerV2(backend_c)
sampler_c.options.dynamical_decoupling.enable = True
sampler_c.options.dynamical_decoupling.sequence_type = "XY4"
sampler_c.options.dynamical_decoupling.skip_reset_qubits = True

In [18]:
job = sampler_c.run(pubs=zip(t_qc_c, params), shots=shots)
print(job.job_id())

d4bnirvnmdfs73ae3a2g


In [19]:
# step 4: post-processing
# job = service.job("d1obp8j3rr0s73bg4810")
result = job.result()
fidelity_c = [fidelity(res) for res in result]
print(fidelity_c)
usage_c = job.usage()

[0.6675, 0.5725, 0.098, 0.102, 0.065, 0.0235, 0.006, 0.0015, 0.0015, 0.002]


## Verglich vun Workflow un Schaltkreis ohne Bruchteil-Gates
En däm Abschnett präsentiere mer dä Standard Qiskit patterns Workflow met enem Backend dat kein Bruchteil-Gates ongerstötz.
Endem de de transpileete Schaltkreis vergliechs, wees de merke dat de Versjohn met Bruchteil-Gates (vum vörijje Abschnett) kompakter es als die ohne Bruchteil-Gates.

In [20]:
plt.plot(qubits, depth_c, "-o", label="no fractional gates")
plt.plot(qubits, depth_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("depth")
plt.title("Comparison of depths")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/ef343a53-1.avif" alt="Output of the previous code cell" />

In [21]:
plt.plot(qubits, duration_c, "-o", label="no fractional gates")
plt.plot(qubits, duration_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("duration (µs)")
plt.title("Comparison of durations")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/98bb2cd0-1.avif" alt="Output of the previous code cell" />

In [22]:
plt.plot(qubits, nnl_c, "-o", label="no fractional gates")
plt.plot(qubits, nnl_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("number of non-local gates")
plt.title("Comparison of numbers of non-local gates")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/1383b242-1.avif" alt="Output of the previous code cell" />

In [23]:
plt.plot(qubits, fidelity_c, "-o", label="no fractional gates")
plt.plot(qubits, fidelity_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("fidelity")
plt.title("Comparison of fidelities")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/8b4594f5-1.avif" alt="Output of the previous code cell" />

We compare the QPU usage time with and without fractional gates. The results in the following cell show that the QPU usage times are almost identical.

In [24]:
print(f"no fractional gates: {usage_c} seconds")
print(f"fractional gates: {usage_f} seconds")

no fractional gates: 7 seconds
fractional gates: 7 seconds


## Advanced topic: Using only fractional RX gates

The need for the modified workflow when using fractional gates primarily stems from the restriction on RZZ gate angles.
However, if you use only the fractional RX gates and exclude the fractional RZZ gates, you can continue to follow the standard Qiskit patterns workflow.
This approach can still offer meaningful benefits, particularly in circuits that involve a large number of RX gates and U gates, by reducing the overall gate count and potentially improving performance.
In this section, we demonstrate how to optimize your circuits using only fractional RX gates, while omitting RZZ gates.

To support this, we provide a utility function that allows you to disable a specific basis gate in a Target object.
Here, we use it to disable RZZ gates.

In [25]:
from qiskit.circuit.library import n_local
from qiskit.transpiler import Target

In [26]:
def remove_instruction_from_target(target: Target, gate_name: str) -> Target:
    new_target = Target(
        description=target.description,
        num_qubits=target.num_qubits,
        dt=target.dt,
        granularity=target.granularity,
        min_length=target.min_length,
        pulse_alignment=target.pulse_alignment,
        acquire_alignment=target.acquire_alignment,
        qubit_properties=target.qubit_properties,
        concurrent_measurements=target.concurrent_measurements,
    )

    for name, qarg_map in target.items():
        if name == gate_name:
            continue
        instruction = target.operation_from_name(name)
        if qarg_map == {None: None}:
            qarg_map = None
        new_target.add_instruction(instruction, qarg_map, name=name)
    return new_target

We use a circuit consisting of U, CZ, and RZZ gates as an example.

In [27]:
qc = n_local(3, "u", "cz", "linear", reps=1)
qc.rzz(1.1, 0, 1)
qc.draw("mpl")

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6b812497-0.avif" alt="Output of the previous code cell" />

We first transpile the circuit for a backend that does not support fractional gates.

In [28]:
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc = pm_c.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 23), ('sx', 16), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/9e8e0709-1.avif" alt="Output of the previous code cell" />

## Verglich vun Deefe un Fidelities
En däm Abschnett verglieche mer de Zahl vun Nit-Lokal Gates un de Fidelities zwesche Schaltkreis met un ohne Bruchteil-Gates.
Dat hält de potenziell Vordeile vun de Verweendung vun Bruchteil-Gates bezöglich Ußföhrungseffizienz un Qualität hörvor.

In [29]:
backend_f = service.backend(backend_name, use_fractional_gates=True)
target = remove_instruction_from_target(backend_f.target, "rzz")
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 22), ('sx', 14), ('cz', 4), ('rx', 1)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/db45feb0-1.avif" alt="Output of the previous code cell" />

### Optimize U gates with fractional RX gates

In this section, we demonstrate how to optimize U gates using fractional RX gates, building on the same circuit introduced in the previous section.

You will need to install the `qiskit-basis-constructor` [package](https://github.com/Qiskit/qiskit-basis-constructor) for this section.
This is a beta version of a new transpilation plugin for Qiskit, which might be integrated into Qiskit in the future.

In [30]:
# %pip install qiskit-basis-constructor

In [31]:
from qiskit.circuit.library import UGate
from qiskit_basis_constructor import DEFAULT_EQUIVALENCE_LIBRARY

We transpile the circuit using only fractional RX gates, excluding RZZ gates.
By introducing a custom decomposition rule, as shown in the following,
we can reduce the number of single-qubit gates required to implement a U gate.

This feature is currently under discussion in this [GitHub issue.](https://github.com/Qiskit/qiskit/issues/13455)

In [32]:
# special decomposition rule for UGate
x = ParameterVector("x", 3)
zxz = QuantumCircuit(1)
zxz.rz(x[2] - np.pi / 2, 0)
zxz.rx(x[0], 0)
zxz.rz(x[1] + np.pi / 2, 0)
DEFAULT_EQUIVALENCE_LIBRARY.add_equivalence(UGate(x[0], x[1], x[2]), zxz)

Next, we apply the transpiler using `constructor-beta` translation provided by `qiskit-basis-constructor` package.
As a result, the total number of gates is reduced compare to the previous transpilation.

In [33]:
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
    translation_method="constructor-beta",
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 16), ('rx', 9), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b19aae7c-1.avif" alt="Output of the previous code cell" />

## Troubleshooting

### Issue: Invalid RZZ angles might remain after transpilation

As of Qiskit v2.0.3, there are known issues where RZZ gates with invalid angles may remain in the circuits even after transpilation.
The issue typically arises under the following conditions.

#### Failure when using `target` option with `generate_preset_pass_manager` or `transpiler`

When the `target` option is used with `generate_preset_pass_manager` or `transpiler`, the specialized transpiler pass `FoldRzzAngle` is not invoked.
To ensure proper handling of RZZ angles for fractional gates, we recommend always using the `backend` option instead.
See [this issue](https://github.com/Qiskit/qiskit/issues/14318) for more details.

#### Failure when circuits contain certain gates

If your circuit includes certain gates such as `XXPlusYYGate`, the Qiskit transpiler may generate RZZ gates with invalid angles.
If you encounter this issue, see this [GitHub issue](https://github.com/Qiskit/qiskit-ibm-runtime/issues/2256#issuecomment-2889487152) for a workaround.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_cCNiGkGX5xZMzoG)